# ![CreditCardImage](./image-112.png)

 # Veri Ön İşleme ve Standartlaştırma


**İşlem tutarları gibi sürekli özelliklerin standartlaştırılması (StandardScaler kullanımı).Log dönüşümü gibi yöntemlerle veri dağılımının düzenlenmesi.
Özellik Mühendisliği:Yeni özellikler türetilmesi (etkileşim özellikleri, kare alma gibi doğrusal olmayan dönüşümler).Özellikler arasındaki yüksek korelasyonların analizi ve gruplanması.Modelleme:Veri kümesinin eğitim ve test setlerine ayrılması.RandomForestClassifier gibi bir modelin kullanılması.
Çapraz doğrulama ve model değerlendirme metrikleri (accuracy, classification_report).Özellik Önemi:Özellik önemlerini analiz eden bir grafik ile önemli değişkenlerin belirlenmesi (plot_importance_ensemble fonksiyonu).**


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
pd.set_option("display.max_columns",None)
import shutil
# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-fraud-detection-dataset-2023/creditcard_2023.csv
/kaggle/input/creditcardimage/image-112.png


In [ ]:
shutil.copy('/kaggle/input/creditcardimage/image-112.png', '/kaggle/working/image-112.png')

# LOAD DATASET

**Veri Setini Yükleme Ve İlk Beş Satır**

In [ ]:
# Veri setini yükleme
df = pd.read_csv("/kaggle/input/credit-card-fraud-detection-dataset-2023/creditcard_2023.csv")

# Veri setinin ilk 5 satırını görüntüleme
df.head()

# Exploratory Data Analysis
**Keşifsel Veri Analizi**

In [ ]:
# Veri setinin boyutları
print(df.shape)

# Eksik değer kontrolü
print(df.isnull().sum())

# Sınıf dağılımı
print(df['Class'].value_counts(normalize=True))


In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.tail()

# Data Visualization

**Veri Görselleştirme**
Sınıf Ve Tutar değişkenlerimizi görselleştiriyoruz.
İki değişkeninde hemen hemen normal dağıldığını söylemek mümkün ve iki değişkenimiz için de ayrkırı değerimiz olmadığı ortada.

In [ ]:
sns.countplot(x="Class", data=df)
plt.title("Sınıf Dağılımı (Fraud vs Not Fraud)")
plt.show()

In [ ]:
sns.histplot(df['Amount'])
plt.title("İşlem Tutarı Dağılımı")
plt.show()


In [ ]:
sns.boxplot(x=df['Amount'])
plt.title("İşlem Tutarı Dağılımı (Boxplot)")
plt.xlabel("İşlem Tutarı")
plt.show()


In [ ]:
sns.violinplot(x=df['Amount'])
plt.title("İşlem Tutarı Dağılımı (Violin Plot)")
plt.xlabel("İşlem Tutarı")
plt.show()


In [ ]:
sns.violinplot(x='Class', y='Amount', data=df, scale='width')
plt.title("Sınıf ve İşlem Tutarı Dağılımı (Violin Plot)")
plt.xlabel("Class (0: Fraud Değil, 1: Fraud)")
plt.ylabel("İşlem Tutarı")
plt.show()

In [ ]:
df['Class'].value_counts()

In [ ]:
df.groupby('Class')['Amount'].mean()

**İşlem tutarlarına göre doladırıcılık  durumuna  bakıtığımızda dolandırma tutarları normal işlem tutarlarına göre dafa fazla miktarda olduğu görünüyor bu miktar az da  olsa önemli bir fark yaratıyor.**

In [ ]:
sns.stripplot(x='Class', y='Amount', data=df, jitter=True, size=2, alpha=0.7)
plt.title("Sınıf ve İşlem Tutarı Dağılımı (Strip Plot)")
plt.xlabel("Class (0: Fraud Değil, 1: Fraud)")
plt.ylabel("İşlem Tutarı")
plt.show()


In [ ]:
g = sns.FacetGrid(df, col="Class", height=5, aspect=1.2, palette="viridis")
g.map(sns.histplot, "Amount", bins=50, kde=True)
g.set_titles("Class {col_name}")
g.fig.suptitle("Sınıflara Göre İşlem Tutarı Dağılımı", y=1.02)
plt.show()


# Veriyi ön işlerken tekrar aykırı değerler ve null değerlerin olup olmadığını gözden geçiriyoruz 

In [ ]:
# Null değerlerin toplamı
print(df.isnull().sum())

# Tutar Değişkeninin z-score ve Iqr yönetmiyle aykırı değer tespiti

In [ ]:
# Belirli bir sütun için IQR hesaplama 
Q1 = df['Amount'].quantile(0.25)
Q3 = df['Amount'].quantile(0.75)
IQR = Q3 - Q1

# Aykırı değer sınırları
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Aykırı değerlerin tespiti
outliers = df[(df['Amount'] < lower_bound) | (df['Amount'] > upper_bound)]
print(f"Aykırı değer sayısı: {len(outliers)}")


In [ ]:
# Tutar sütunu için Z-Score
df['z_score'] = zscore(df['Amount'])

# Z-Score > 3 veya < -3 olanlar
outliers_z = df[(df['z_score'] > 3) | (df['z_score'] < -3)]
print(f"Z-Score yöntemiyle tespit edilen aykırı değer sayısı: {len(outliers_z)}")


**sınıf değişkeni herhangi bir aykırı değer olmamasıyla beraber  normal bir dağılım seyretmiş diyebiliriz**

In [ ]:
sns.boxplot(x='Class', y='Amount', data=df)
plt.title("Classlara göre İşlem Tutarlarının Box Plot Grafiği")
plt.xlabel("Class (0: Fraud Değil, 1: Fraud)")
plt.ylabel("İşlem Tutarı")
plt.show()


# Veri Setimizi Güvene Alıyoruz

In [ ]:
df= df.drop('z_score',axis=1)

In [ ]:
data = df.copy()

In [ ]:
df.head()

In [ ]:
df1 = df.copy()

In [ ]:
df1.head()

# Feature Engineering(Özellik Mühendisliği)
**Feature engineering (özellik mühendisliği), makine öğrenimi modellerinin performansını artırmak ve daha iyi tahminler yapmasını sağlamak için veri setindeki özellikleri dönüştürme, yeni özellikler oluşturma veya mevcutları yeniden düzenleme işlemidir.**

Model Performansını Artırmak: Daha anlamlı ve iyi tasarlanmış özellikler, modelin doğruluğunu artırır.
Veri Setindeki Eksiklikleri Gidermek: Eksik veya hatalı verileri temizlemek ve dönüştürmek gerekir.
Aykırı Değerlerin Etkisini Azaltmak: Aykırı değerleri işlemek modeli daha kararlı hale getirir.
Karmaşık İlişkileri Ortaya Çıkarmak: Özellikler arasındaki ilişkileri keşfetmek ve model için daha anlamlı hale getirmek.
Özelliklerin Ölçeklendirilmesi: Modelin hızlı ve etkili öğrenmesini sağlamak için veriler aynı ölçeklerde olmalıdır.


**Modelde Kullanılmayacak Değişkenlerin Kaldırılması**

In [ ]:
df.columns

**id sütunun modelimizde herehangi bir analam ifade etmeyeceğinden kaldırdık**

In [ ]:
df = df.drop(columns=['id'])

**İşlem Tutarını Standartlaştırılması**

In [ ]:
scaler = StandardScaler()
df['Amount'] = scaler.fit_transform(df[['Amount']])

In [ ]:
df.head()

**Daha güçlü  model için güçlü bir bir özellik türetmek için mevcut sütunları analiz edip, yeni bir özellik oluşturuyoruz.**

**Etkileşim Özelliği Türetme**

In [ ]:
df['V1_Amount_Interaction'] = df['V1'] * df['Amount']

**Doğrusal Olmayan Özellikler Ekleme**

In [ ]:
df['V1_Squared'] = df['V1'] ** 2

**Log Transformation ile Yeni Özellik**

In [ ]:
df['Log_Amount'] = np.log1p(df['Amount'])

**Standartlaştırılmış Özellik**

In [ ]:

df['V1_Amount_Interaction_Scaled'] = scaler.fit_transform(df[['V1_Amount_Interaction']])


**Korelasyon Analizi**

In [ ]:
correlation = df[['Class', 'V1_Amount_Interaction', 'Log_Amount']].corr()
sns.heatmap(correlation, annot=True, cmap="coolwarm")
plt.title("Yeni Özelliklerin Korelasyonu")
plt.show()

**Özellik Kombinasyonları**

In [ ]:
df['V1_V2_Interaction'] = df['V1'] * df['V2']
df['V3_V4_Interaction'] = df['V3'] * df['V4']

 **İstatistiksel Özellikler Türetme**

In [ ]:
df['Mean_V1_V2'] = df[['V1', 'V2']].mean(axis=1) #ortlama 

In [ ]:
df['Std_V1_V2'] = df[['V1', 'V2']].std(axis=1) #standart sapma

In [ ]:
df['Var_V1_V2'] = df[['V1', 'V2']].var(axis=1) #varyans

In [ ]:
df.head()

**Anonimleştirilmiş Özelliklerin Korelasyonu**

In [ ]:
df.columns

In [ ]:

# Korelasyon matrisini hesaplayalım
correlation = df[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
                  'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
                  'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']].corr()

# Korelasyon matrisini ısı haritası olarak görselleştirelim
plt.figure(figsize=(12, 10))  # Grafik boyutunu ayarlayalım
sns.heatmap(correlation, annot=True, cmap="coolwarm", fmt='.2f', linewidths=0.5)

# Başlık ve etiketler
plt.title("Anonimleştirilmiş Özelliklerin Korelasyonu")
plt.tight_layout()

# Görselleştirmeyi gösterelim
plt.show()


In [ ]:

correlation = df[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
                  'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
                  'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']].corr()

# Korelasyon değerinin mutlak değeri 0.80'den büyük olanları bulalım
high_corr = correlation[correlation.abs() > 0.80]

# 1.0 korelasyonunu çıkaralım çünkü bu, değişkenlerin kendileriyle olan korelasyonudur
high_corr = high_corr[high_corr < 1.0]

# Korelasyonu 0.80'den büyük olan değişken çiftlerini göstermek
high_corr_pairs = high_corr.stack().reset_index()
high_corr_pairs.columns = ['Feature 1', 'Feature 2', 'Correlation']

# Yüksek korelasyonlu değişken çiftlerini yazdıralım
print("Yüksek Korelasyonlu Değişken Çiftleri:")
print(high_corr_pairs)

# Maskelenmiş korelasyon matrisini görselleştirelim
plt.figure(figsize=(12, 10))  # Grafik boyutunu ayarlayalım
sns.heatmap(correlation, annot=True, cmap="coolwarm", fmt='.2f', linewidths=0.5, mask=~(high_corr.abs() > 0.80))

# Başlık ve etiketler
plt.title("Korelasyonu 0.80'den Yüksek Olan Özellikler")
plt.tight_layout()

# Görselleştirmeyi gösterelim
plt.show()

**V16,V17  VE V17,V18 DEĞİŞKENLERİNİ KORELASYONLARI YÜKSEK ÇIKTIĞINDAN GRUPLANDIRALIM**

In [ ]:
# Korelasyonu yüksek olan değişkenleri gruplayalım

# V16 ve V17'nin ortalamasını alalım
df['V16_V17_Group'] = (df['V16'] + df['V17']) / 2

# V17 ve V18'in ortalamasını alalım
df['V17_V18_Group'] = (df['V17'] + df['V18']) / 2

# İlk birkaç satırı kontrol edelim
print(df[['V16', 'V17', 'V18', 'V16_V17_Group', 'V17_V18_Group']].head())

In [ ]:
df.head()

In [ ]:
df.columns

# Diğer Değişkenlerin biribiri Arasında Korelasyonları

In [ ]:
# İlgili değişkenler
selected_columns = ['Amount', 'Class', 'V1_Amount_Interaction', 'Log_Amount',
                    'V1_Amount_Interaction_Scaled', 'V1_V2_Interaction', 'V3_V4_Interaction',
                    'Mean_V1_V2', 'Std_V1_V2', 'Var_V1_V2', 'V16_V17_Group', 'V17_V18_Group']

# Korelasyon matrisini hesaplayalım
correlation_matrix = df[selected_columns].corr()

# Korelasyon matrisini görselleştirelim
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt='.2f')
plt.title('Korelasyon Matrisi')
plt.show()

# Yüksek korelasyonları (0.8 ve üzeri) filtreleyelim
high_correlation = correlation_matrix[correlation_matrix.abs() > 0.8]

# Yüksek korelasyonları yazdıralım
print(high_correlation)


In [ ]:
# Yüksek korelasyona sahip olan sütunları silme
df = df.drop(columns=['V1_Amount_Interaction_Scaled', 'V16_V17_Group','V1_Squared', 'Log_Amount','V17_V18_Group','V17'])

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), cmap='coolwarm')
plt.title('Korelasyon Matrisi')
plt.show()

# MODEL SELECTİON(MODEL KURULUMU)

In [ ]:

# Özellikleri ve hedefi ayırma
X = df.drop(columns=['Class'],axis=1)
y = df['Class']

# Eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)



In [ ]:
# Model oluşturma
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Tahmin ve değerlendirme
y_pred = model.predict(X_test)

# Model performansını yazdırma
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print(accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Modelinizi tanımlayın
rf = RandomForestClassifier(random_state=42)

# Çapraz doğrulama ile modelin doğruluğunu değerlendirin
cv_scores = cross_val_score(rf, X, y, cv=3, scoring='accuracy')

# Çapraz doğrulama sonuçları
print(f'Çapraz doğrulama ile doğruluk skorları: {cv_scores}')
print(f'Ortalama doğruluk: {np.mean(cv_scores)}')
print(f'Karmaşıklık: {np.std(cv_scores)}')


In [ ]:
def plot_importance_ensemble(model, features, num=len(X)):
    
    feature_imp_df = pd.DataFrame(0, index=features.columns, columns=["Value"])
    
    
    for model in model.estimators_:
        if hasattr(model, 'feature_importances_'):
            
            feature_imp_df["Value"] += model.feature_importances_/100

    
    feature_imp_df["Feature"] = features.columns
    feature_imp_df = feature_imp_df.sort_values(by="Value", ascending=False)
    plt.figure(figsize=(12, 7))
    sns.set(font_scale=1)
    ax = sns.barplot(x="Value", y="Feature", data=feature_imp_df[0:num])
    
    for p in ax.patches:
        ax.annotate(format(p.get_width(), '.4f'),
                    (p.get_width(), p.get_y() + p.get_height() / 2),
                    ha='left', va='center', fontsize=8)
    
    plt.title("Feature Importance -  Classifier")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_importance_ensemble(model,X)

**Modelimizde en  çok tahmin skorunu etkileyen özellik V14 olmuştur bu sebeple V14  bizim için Frud tahmini sürecinde  kritik rol oynamaktadır bunun yanı sıra V12,V4,V11 özellikleri de farud izlenim sürecinde önemli bir özellik göstermektedir.**